<a href="https://colab.research.google.com/github/ownit4137/TIL/blob/main/CustomDataset/210204_CarPlateOcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CRNN + OCR

- https://github.com/qjadud1994/CRNN-Keras

In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/CNNprac

!git clone https://github.com/qjadud1994/CRNN-Keras

/content/drive/MyDrive/Colab Notebooks/CNNprac
Cloning into 'CRNN-Keras'...
remote: Enumerating objects: 138, done.
remote: Total 138 (delta 0), reused 0 (delta 0), pack-reused 138
Receiving objects: 100% (138/138), 385.20 KiB | 11.67 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data

/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data


- csv 파일을 불러와 dataframe으로 저장

In [4]:
import pandas as pd

img_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/groundtruth.csv', names=['src', 'state', 'plate'], header=None)

In [5]:
img_info

,src,state,plate
0,ak1165.png,ak,FUW999
1,ak399.png,ak,FGJ235
2,ak721.png,ak,FHG521
3,ak848.png,ak,FPJ331
4,al1156.png,al,6A5730A
...,...,...,...
746,wy508.png,wy,3555
747,wy658.png,wy,4475
748,wy712.png,wy,4779
749,wy716.png,wy,79XA


- 이미지 파일의 이름이 답이어야 하므로 수정
- test 데이터 복사

In [40]:
import os
import numpy as np
import random
import shutil

# for src, state, plate in img_info.values:
  # os.rename('/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/images/' + src, '/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/images/' + plate + '.png')

# for i in range (10):
#   a = np.random.randint(0, 751)
#   randsrc = img_info.iloc[a]['plate'] + '.png'
#   shutil.copy('/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/images/' + randsrc, '/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras/DB/test/' + randsrc)

In [39]:
%cd /content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/images
%cd /content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras/DB/test

!rm -r .ipynb_checkpoints

/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/images
/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras/DB/test
rm: cannot remove '.ipynb_checkpoints': No such file or directory


In [43]:
import torch

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True


print(torch.cuda.is_available())

!nvidia-smi # gpu 확인

True
Wed Feb  3 14:36:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

# Training

In [71]:
%cd '/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras'

!python training.py

/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras
2021-02-03 15:41:13.265744: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2021-02-03 15:41:15.278903: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-03 15:41:15.279932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-03 15:41:15.286142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysF

# loss

- 각 에폭마다의 마지막 모델을 저장
- **300에폭** :  80 -> 17
- **200에폭** :  17 -> 6
- **200에폭** :  6 -> 3

In [73]:
!ls /content

drive			   LSTM+BN5--300--17.111.hdf5  sample_data
LSTM+BN5--200--3.272.hdf5  LSTM+BN5--300--5.803.hdf5   saved_models


### Test

- 랜덤으로 선택한 10가지 이미지에 대해 prediction 수행

```
Predicted: LMS1  /  True: LMS301
Predicted: TFD715  /  True: TFD715
Predicted: 093GOK  /  True: 093GOK
Predicted: 1NU656  /  True: 1NU656
Predicted: KLX41C  /  True: KLX41C
Predicted: 602JP  /  True: 602JPG
Predicted: RXH400  /  True: RXH400
Predicted: 017M149  /  True: 017M149
Predicted: KRM489  /  True: KRM489
Predicted: 692  /  True: 692
Time :  0.3941304683685303
ACC :  0.8
letter ACC :  0.9310344827586207
```

In [74]:
!python '/content/drive/MyDrive/Colab Notebooks/CNNprac/carplate_ocr_data/CRNN-Keras/Prediction.py' --weight /content/LSTM+BN5--200--3.272.hdf5

2021-02-03 15:53:00.037618: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2021-02-03 15:53:02.475143: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-03 15:53:02.484112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-02-03 15:53:02.489117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

### accuracy : 93%